In [1]:
from scipy.optimize import curve_fit
import pandas as pd
import random
import sys
from IPython.display import clear_output
from matplotlib import pyplot as plt

sys.path.append('../..')

from model import run
from model import plot_utils as pu
from model import processing
from model.amm.omnipool_amm import OmnipoolState
from model.amm.agents import Agent
from model.amm.trade_strategies import toxic_asset_attack
from model.amm.global_state import GlobalState

In [2]:
# same seed, same parameters = same simulation result
random.seed(42)

assets = {
    'HDX': {'usd price': 0.05, 'weight': 0.10},
    'DAI': {'usd price': 1, 'weight': 0.20},
    'AUSD': {'usd price': 1, 'weight': 0.10},
    'ETH': {'usd price': 2500, 'weight': 0.40},
    'DOT': {'usd price': 5.37, 'weight': 0.20}
}
lrna_price_usd = 0.07

initial_omnipool_tvl = 10000000

def lrna_price(asset: str):
    return price_dict['LRNA'] / price_dict[asset]

liquidity = {}
lrna = {}
for tkn, info in assets.items():
    liquidity[tkn] = initial_omnipool_tvl * info['weight'] / info['usd price']
    lrna[tkn] = initial_omnipool_tvl * info['weight'] / lrna_price_usd
    
print(liquidity)
print(lrna)

initial_state = GlobalState(
    pools={
        'omnipool': OmnipoolState(
            tokens={
                tkn: {'liquidity': liquidity[tkn], 'LRNA': lrna[tkn]} for tkn in assets
            },
            lrna_fee=0,
            asset_fee=0,
            preferred_stablecoin = 'DAI'
        )
    },
    agents={
        'Attacker': Agent(
            holdings={'USD': 0, 'shitcoin': float('inf')},
            trade_strategy=toxic_asset_attack(
                pool_id='omnipool',
                asset_name='shitcoin',
                trade_size=10000
            )
        )
    },
    external_market={tkn: assets[tkn]['usd price'] for tkn in assets}
)
print(initial_state)

{'HDX': 20000000.0, 'DAI': 2000000.0, 'AUSD': 1000000.0, 'ETH': 1600.0, 'DOT': 372439.47858472995}
{'HDX': 14285714.285714284, 'DAI': 28571428.571428567, 'AUSD': 14285714.285714284, 'ETH': 57142857.142857134, 'DOT': 28571428.571428567}
global state 
pools: 

    Omnipool: omnipool
    ********************************
    tvl cap: inf
    lrna fee: 0%
    asset fee: 0%
    asset pools: (
    
        *HDX*
        asset quantity: 20000000.0
        lrna quantity: 14285714.285714284
        USD price: 0.05
        tvl: 1000000.0
        weight: 14285714.285714284/142857142.85714284 (0.1)
        weight cap: 1.0
        total shares: 20000000.0
        protocol shares: 20000000.0
    
        *DAI*
        asset quantity: 2000000.0
        lrna quantity: 28571428.571428567
        USD price: 1.0
        tvl: 2000000.0
        weight: 28571428.571428567/142857142.85714284 (0.2)
        weight cap: 1.0
        total shares: 2000000.0
        protocol shares: 2000000.0
    
        *AUSD*
  

In [ ]:
%matplotlib inline